In [ ]:
# Import packages
import pandas as pd
import numpy as np
from numbers import Number
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [ ]:
# View directory
!ls

In [ ]:
# Import AviationData file
aviation = pd.read_csv('AviationData.csv', encoding='ISO-8859-1', parse_dates=['Event.Date', 'Publication.Date'])

In [ ]:
# Import USState_Codes file
us_state_codes = pd.read_csv('USState_Codes.csv')
us_state_codes.head()

In [ ]:
# Explore data
aviation.head()

In [ ]:
aviation.tail()

In [ ]:
aviation.describe()

In [ ]:
aviation.info()

In [ ]:
aviation.isna().sum()

In [ ]:
# Found 938 duplicate Event.Ids
aviation.duplicated(subset='Event.Id').value_counts()

In [ ]:
# Drop duplicate Event.Ids
aviation.drop_duplicates(subset='Event.Id', inplace=True)

In [ ]:
"""
Aircraft accident means an occurrence associated with the operation of an aircraft which takes place between the time any person boards the aircraft with the intention of flight and all such persons have disembarked, and in which any person suffers death or serious injury, or in which the aircraft receives substantial damage

Incident means an occurrence other than an accident, associated with the operation of an aircraft, which affects 

Source: https://www.govinfo.gov/content/pkg/CFR-2011-title49-vol7/pdf/CFR-2011-title49-vol7-sec830-2.pdf
"""

aviation['Investigation.Type'].value_counts()

In [ ]:
aviation['Accident.Number'].value_counts()

In [ ]:
aviation.duplicated(subset='Accident.Number').value_counts()

In [ ]:
# Split out the event date into year, month and day using DateTime methods

aviation['Event.Year'] = aviation['Event.Date'].dt.year
aviation['Event.Month'] = aviation['Event.Date'].dt.month
aviation['Event.Day'] = aviation['Event.Date'].dt.day

In [ ]:
# Create new dataframe of incidents by year
incidents_by_year = aviation['Event.Year'].value_counts()
incidents_by_year = incidents_by_year.sort_index()
incidents_by_year

In [ ]:
# Graph incidents by year
years = list(incidents_by_year.index)
incidents = list(incidents_by_year.values)

fig, ax = plt.subplots(figsize=(16, 12))

ax.bar(years, incidents)
ax.set_title("Aircraft Incidents by Year")
ax.set_ylabel("Number of Incidents")
ax.set_xlabel("Years")


In [ ]:
"""
Maddie: The number of airplane accidents by year rise until about 2009 then it remains relatively steady. I found an article that seems to back this up, I think it could be a good cut-off point!
https://www.cnbc.com/2019/02/13/colgan-air-crash-10-years-ago-reshaped-us-aviation-safety.html
"""

# Filter only data after 2009
# TODO: LL: Let's discuss the year cutoff point again as I'm not seeing the 2009 drop in the data. Was this after cleaning everything else
aviation = aviation[aviation['Event.Year'] > 2009]

In [ ]:
aviation.loc[(aviation['Country'] != 'United States')]

In [ ]:
# Parse records for State column from records in Location column
aviation['State'] = aviation['Location'].str.split(", ").str[1]

In [ ]:
# Update state records for non-US locations
aviation.loc[aviation['Country'] != 'United States', 'State'] = 'non-US'

In [ ]:
# Review State records
aviation['State'].value_counts()

In [ ]:
# Review State records that need to be cleaned
aviation['State'].value_counts(sort=True, ascending=True)[:25]

In [ ]:
# 1 record that is AO
aviation.loc[(aviation['Location'] == 'Atlantic Ocean')]

# 1 record that is AS
aviation.loc[(aviation['State'] == "MANU'A")]

# 1 record that is CA
aviation.loc[(aviation['State'] == 'LA,')]

# 2 records that are Unknown and 1 that is GM
aviation.loc[(aviation['State'] == 'UN')]

# 16 records that are HI
aviation.loc[(aviation['State'] == 'MAUI') | (aviation['State'] == 'Maui') | (aviation['State'] == 'OAHU') | (aviation['State'] == 'KAUAI') | (aviation['State'] == 'HONOLULU')]

# records that are non-US
aviation.loc[(aviation['State'] == 'MG') | (aviation['State'] == 'OF')]

# 1 record that is NY
aviation.loc[(aviation['State'] == 'NYC')]

# 1 record that is PR
aviation.loc[(aviation['State'] == 'San Juan Is.')]

# 1 record that is VI
aviation.loc[(aviation['State'] == 'CB')]

In [ ]:
# Change record to AO
aviation.loc[(aviation['Location'] == 'Atlantic Ocean, OF'), 'State'] = 'AO'

# Change record to AS
aviation.loc[(aviation['State'] == "MANU'A"), 'State'] = 'AS'

# Change record to CA
aviation.loc[(aviation['State'] == 'LA,'), 'State'] = 'CA'

# Change record to GM
aviation.loc[(aviation['Location'] == 'GULF OF MEXICO, UN'), 'State'] = 'GM'

# Change records to HI
aviation.loc[(aviation['State'] == 'MAUI') | (aviation['State'] == 'Maui') | (aviation['State'] == 'OAHU') | (aviation['State'] == 'Oahu') | (aviation['State'] == 'KAUAI') | (aviation['State'] == 'HONOLULU'), 'State'] = 'HI'

# # Change record to non-US
aviation.loc[((aviation['State'] == 'MG') | (aviation['State'] == 'OF')) & (aviation['Location'] != 'Atlantic Ocean, OF'), 'State'] = 'non-US'

# Change record to NY
aviation.loc[(aviation['State'] == 'NYC'), 'State'] = 'NY'

# Change record to PR
aviation.loc[(aviation['State'] == 'San Juan Is.'), 'State'] = 'PR'

# Change record to Unknown
aviation.loc[(aviation['Location'] == 'Missing, UN') | (aviation['Location'] == 'Oceanic, UN'), 'State'] = 'Unknown'

# Change record to VI
aviation.loc[(aviation['State'] == 'CB'), 'State'] = 'VI'

In [ ]:
# Verify all State records are cleaned
aviation['State'].value_counts()

In [ ]:
# Verify only Unknown and/or non-US state locations appear in cleaned State records
invalid_state_locations = []

for state in list(aviation['State'].value_counts().index):
  if state not in list(us_state_codes['Abbreviation']):
    invalid_state_locations.append(state)

invalid_state_locations

In [ ]:
aviation['Country'].isna().value_counts()

In [ ]:
blank_countries = set(aviation.loc[(aviation['Country'].isna())]['Location'])
blank_countries
len(blank_countries) # 0 if years are 2009+, otherwise blank countries need to be scrubbed

In [ ]:
# Fix Country for US locations with blank Country records

# TODO: LL: Check more locations if year cutoff is prior to 2009

valid_US_locations = [
 'CHARLOTTE AMALI',
 'CHRISTIANSTED',
'FAJARDO',
 'GUAM',
'GUAYAMA',
'Guam',
'NEAR SAN JUAN',
 'NEAR ST. CROIX',
 'NO.ATLANTIC OC',
 'SAINT CROIX',
 'ST CROIX',
 'ST THOMAS',
 'ST. CROIX',
 'ST. THOMAS',
 'ST.THOMAS',
 'STAKE ISLAND',
 'VIEQUES']

aviation.loc[(aviation['Location'].isin(valid_US_locations)), 'Country'] = 'United States'

In [ ]:
# Fix State for US locations with blank Country records

# TODO: LL: Check more locations if year cutoff is prior to 2009

valid_VI_locations = [
'CHARLOTTE AMALI',
 'CHRISTIANSTED',
'NEAR ST. CROIX',
'SAINT CROIX',
 'ST CROIX',
 'ST THOMAS',
 'ST. CROIX',
 'ST. THOMAS',
'ST.THOMAS',
'VIEQUES']

valid_PR_locations = [
'FAJARDO',
'GUAYAMA',
'NEAR SAN JUAN']

valid_GM_locations = [
 'GUAM',
'Guam']

valid_AO_locations = [
'NO.ATLANTIC OC']

valid_MS_locations = [
'STAKE ISLAND']

aviation.loc[(aviation['Location'].isin(valid_VI_locations)), 'State'] = 'VI'
aviation.loc[(aviation['Location'].isin(valid_PR_locations)), 'State'] = 'PR'
aviation.loc[(aviation['Location'].isin(valid_GM_locations)), 'State'] = 'GM'
aviation.loc[(aviation['Location'].isin(valid_AO_locations)), 'State'] = 'SO'
aviation.loc[(aviation['Location'].isin(valid_MS_locations)), 'State'] = 'MS'



In [ ]:
aviation['Amateur.Built'].value_counts()

In [ ]:
# Remove amateur built planes
aviation = aviation[aviation['Amateur.Built'] == 'No']

In [ ]:
# Drop irrelevant columns
aviation.drop(columns=['Accident.Number', 'Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Registration.Number', 'Amateur.Built', 'FAR.Description', 'Schedule', 'Publication.Date', 'Air.carrier'], inplace=True)

In [ ]:
# Note: We no longer have records with format 'Fatal(#)' after dropping records with Event.Year prior to 2009
aviation['Injury.Severity'].value_counts()

In [ ]:
aviation['Aircraft.damage'].value_counts()

In [ ]:
aviation['Aircraft.Category'].value_counts()

In [ ]:
# Impute NaN values in Aircraft Category to "airplane"
aviation['Aircraft.Category'].fillna('Airplane', inplace=True)

In [ ]:
# Filter only airplanes
aviation = aviation[aviation['Aircraft.Category'] == 'Airplane']

In [ ]:
aviation['Make'].value_counts()

In [ ]:
# Convert all rows in column 'Make' to title case strings for easier cleaning
aviation['Make'] = aviation.Make.astype(str).str.title()

In [ ]:
# Convert all rows in column 'Make' to objects
aviation['Make'] = aviation['Make'].astype(object)

In [ ]:
# Boeing is listed in several different formats in the data i.e 'The Boeing Company'. This code checks all rows in 'Make' for substring
# 'boeing' and changes it the value to standard 'boeing'

aviation['Make'].loc[aviation['Make'].str.contains('Boeing')] = 'Boeing'

In [ ]:
def normalize_company_names(df, column_name, company_name):
    '''Takes in df dataframe, checks every value in column_name for substring company_name
        If substring company_name exists, the value of the row is overwritten to company_name.
        Then returns the new df dataframe'''
    df[column_name].loc[df[column_name].str.contains(company_name)] = company_name
    return df

In [ ]:
# Check function by normalizing all companies with 'Piper'
normalize_company_names(aviation, 'Make', 'Piper')

In [ ]:
# Normalize the company names of the top airline manufacturers
normalize_company_names(aviation, 'Make', 'Boeing')
normalize_company_names(aviation, 'Make', 'Airbus')
normalize_company_names(aviation, 'Make', 'Cessna')
normalize_company_names(aviation, 'Make', 'Beech')
normalize_company_names(aviation, 'Make', 'Cirrus')

In [ ]:
# Look at the top 20 companies by number of incidents
aviation['Make'].value_counts()[0:20]

In [ ]:
aviation['Model'].value_counts()

In [ ]:
# TODO: LL: Should we keep records with blank Models?
aviation['Model'].isna().value_counts()

In [ ]:
aviation['Number.of.Engines'].value_counts()

In [ ]:
# TODO: LL: Should we keep records with blank Number.of.Engines?
aviation['Number.of.Engines'].isna().value_counts()

In [ ]:
aviation['Engine.Type'].value_counts()

In [ ]:
# TODO: LL: Should we keep records with blank Engine.Type?
aviation['Engine.Type'].isna().value_counts()

In [ ]:
aviation['Purpose.of.flight'].value_counts()

In [ ]:
aviation['Total.Fatal.Injuries'].value_counts()

In [ ]:
aviation['Total.Fatal.Injuries'].isna().value_counts()

In [ ]:
aviation['Total.Minor.Injuries'].value_counts()

In [ ]:
aviation['Total.Minor.Injuries'].isna().value_counts()

In [ ]:
aviation['Total.Serious.Injuries'].value_counts()

In [ ]:
aviation['Total.Serious.Injuries'].isna().value_counts()

In [ ]:
aviation['Total.Uninjured'].value_counts()

In [ ]:
aviation['Total.Uninjured'].isna().value_counts()

In [ ]:
# Add total passengers column 
aviation['Total.Passengers'] = aviation['Total.Fatal.Injuries'] + aviation['Total.Minor.Injuries'] + aviation['Total.Serious.Injuries'] + aviation['Total.Uninjured']

In [ ]:
aviation['Total.Passengers'].value_counts()

In [ ]:
# Add percent fatalities column
aviation['Percent.Fatalities'] = aviation['Total.Fatal.Injuries'] / aviation['Total.Passengers']

In [ ]:
aviation['Percent.Fatalities'].value_counts()

In [ ]:
# Create a column for the percent of passengers killed or injured in accident 
aviation['Percent.Passengers.Harmed'] = (aviation['Total.Fatal.Injuries'] + aviation['Total.Minor.Injuries'] + aviation['Total.Serious.Injuries']) / aviation['Total.Passengers']

In [ ]:
aviation['Percent.Passengers.Harmed'].value_counts()

In [ ]:
"""
VMC stands for “visual meteorological conditions”. VMC conditions describe the weather conditions and situations in which a pilot can sufficiently and safely maintain visuals of all other aircraft and the terrain around them. 
IMC is “instrument meteorological conditions”. MC conditions describe a situation in which visibility and the weather require the pilot to rely on their instruments, and thus conduct an IFR flight.
IFR stands for “instrument flight rules” and are utilized in situations where the weather or visibility is poor enough to warrant relying on the aircraft instruments only to maintain a safe flight. With both rule sets, the weather will decide what the pilot uses.
Source: https://www.aerospaceexchange.com/blog/what-do-vmc-and-imc-mean-to-the-pilot
"""

aviation['Weather.Condition'].value_counts()


In [ ]:
aviation['Weather.Condition'].isna().value_counts()

In [ ]:
aviation['Broad.phase.of.flight'].value_counts()
aviation['Broad.phase.of.flight'].describe()

In [ ]:
# Drop 'Broad.Phase.of.Flight' column as there were too many null values and there are no records from 2009 and beyond
aviation.drop(columns=['Broad.phase.of.flight'], inplace=True)

In [ ]:
# TODO: LL: If time permits, we could gain additional insight by parsing records in the Report.Status column
aviation['Report.Status'].value_counts()

In [ ]:
aviation.describe()

In [ ]:
aviation.info()

In [ ]:
aviation.head()